In [29]:
import pandas as pd
import requests
import json
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

In [37]:
zone1_training_data = pd.read_csv("seismic_zone_1.csv")
zone1_val_data = pd.read_csv("seismic_zone_1_val.csv")

In [31]:
FEATURES = ["depth", "mag", "gap", "dmin", "rms"]
X_test = zone1_val_data[FEATURES] 
y_true = zone1_val_data[["latitude", "longitude"]]

In [32]:
API_URL = "https://fedml.onrender.com/predict"

predictions = []

for _, row in X_test.iterrows():
    payload = row.to_dict()
    
    response = requests.post(API_URL, json=payload)
    
    if response.status_code == 200:
        pred = response.json()["prediction"]
        predictions.append(pred[0])
    else:
        print(f"Error for input {row}: {response.text}")

In [33]:
print(predictions)

[[9.104788999999997, 126.30407699999991], [9.046328, 126.55313699999995], [8.464489999999998, 126.89308100000005], [11.474763999999999, 123.288332], [10.214765000000007, 124.19327299999986], [12.203899999999997, 123.89699900000001], [7.967854, 125.26291499999999], [9.173031, 126.54832099999994], [10.935919000000004, 124.91359200000001], [10.944843999999998, 124.93166500000005], [11.284814999999998, 124.55103300000005], [7.950206999999997, 125.35398200000004], [11.046110000000004, 125.196334], [9.737944999999995, 125.08066299999994], [10.351885999999995, 125.87100900000004], [10.858643000000002, 124.594634], [8.008827000000005, 125.98612999999992], [7.874467000000002, 126.23150300000005], [7.956059000000006, 126.09881800000002], [9.108523000000002, 125.43237199999997], [8.878331999999999, 126.37440199999986], [11.281902999999991, 124.34894699999995], [9.824016999999994, 125.61612199999992], [8.372584000000003, 126.98547800000001], [10.761912999999998, 124.93298699999993], [11.4156600000

In [34]:
y_pred = pd.DataFrame(predictions, columns=["latitude", "longitude"])

In [ ]:
# Ensure y_pred and y_true are properly formatted
y_pred = pd.DataFrame(predictions, columns=["latitude", "longitude"])
y_true = y_true.reset_index(drop=True)

# Compute Mean Squared Error (MSE)
mse_lat = mean_squared_error(y_true["latitude"], y_pred["latitude"])
mse_lon = mean_squared_error(y_true["longitude"], y_pred["longitude"])

# Compute R² Score
r2_lat = r2_score(y_true["latitude"], y_pred["latitude"])
r2_lon = r2_score(y_true["longitude"], y_pred["longitude"])

# Compute Mean Absolute Error (MAE)
mae_lat = mean_absolute_error(y_true["latitude"], y_pred["latitude"])
mae_lon = mean_absolute_error(y_true["longitude"], y_pred["longitude"])

# Print results
print(f"Mean Squared Error (Latitude): {mse_lat}")
print(f"Mean Squared Error (Longitude): {mse_lon}\n")

print(f"R² Score (Latitude): {r2_lat}")
print(f"R² Score (Longitude): {r2_lon}\n")

print(f"Mean Absolute Error (Latitude): {mae_lat}")
print(f"Mean Absolute Error (Longitude): {mae_lon}")

Mean Squared Error (Latitude): 55.6867846556177
Mean Squared Error (Longitude): 22.85913167833489

R² Score (Latitude): -5.6006510125206415
R² Score (Longitude): -21.468817233732548

Mean Absolute Error (Latitude): 6.683537563025212
Mean Absolute Error (Longitude): 4.655139546218472


In [38]:
import pandas as pd
import requests

API_URL = "https://fedml.onrender.com/retrain"

# Convert DataFrame to JSON
data_payload = {
    "features": zone1_training_data[["depth", "mag", "gap", "dmin", "rms"]].to_dict(orient="records"),
    "latitude": zone1_training_data["latitude"].tolist(),
    "longitude": zone1_training_data["longitude"].tolist()
}

# Query parameter for filename
params = {"save_as": "zone_0_zone_1_training.pkl"}

# Send POST request
response = requests.post(API_URL, params=params, json=data_payload)

# Print server response
print(response.json())

{'message': 'Model retrained and saved as zone_0_zone_1_training.pkl.'}
